# Combine Zooniverse workflow-specific classification and subject records into a single CSV with anonymised PII

This Notebook is designed for Zooniverse project owners who want to process classification (annotation) data and subject records in a single CSV file. It will also replace volunteer usernames, IP addresses and user IDs with a pseudononymous identifier, so that the CSV file can be publically shared without personally identifiable information (PII).

The Notebook can be used with workflow-specific classification files or the general classification file which contains annotations from all tasks. This code is not optimised for very large files but has been tested on files with c 8000 classifications.

Start by exporting your classification and subject files from Zooniverse, and saving them somewhere that the Notebook can access them (see below).

This Notebook reads Zooniverse classification and subject CSV files and performs the following tasks:

1. It combines the information from:  
    (a) the **classifications** file, which contains all the annotations users have provided for a given workflow's subjects, with  
    (b) the information from the **subjects** file, which documents the information about the subjects (items) annotated in specific tasks.

2. It processes the columns that contain information in JSON format and transforms them (and the rest of the data) into spreadsheet columns for ease of use. 

3. It removes any Personally Identifiable Information, such as usernames and IP addresses.

4. It produces a .csv file that can be opened in any spreadsheet tool (Microsoft Excel, Apple Numbers, etc.) for further processing.

## Getting set up

In the next cell, you will specify where the data that you want to process is located. These should be two paths on your drive, which can be _relative_ (to where this notebook is located) or _absolute_ (a specified full path to each file).

#### Example 1: Sample data

If you do not have any downloaded data, but want to use the sample data that we have provided (located in the same folder as this notebook), you should put these two _relative_ paths in the next cell:

```py
classifications_file = "sample_data/test_classifications.csv"
subjects_file = "sample_data/test_subjects.csv"
```

Pandas is a very flexible Python package, which can also accept valid URLs, so you could also write:

```py
classifications_file = "https://raw.githubusercontent.com/Living-with-machines/zooniverse-analysis-workshop/main/sample_data/test_classifications.csv"
subjects_file = "https://raw.githubusercontent.com/Living-with-machines/zooniverse-analysis-workshop/main/sample_data/test_subjects.csv"
```

#### Example 2: Downloaded data in Downloads folder

If you have downloaded the files as `classifications.csv` and `subjects.csv` on a Mac and they are located in your `Downloads` folder, in the next cell you would put two _absolute_ paths:

```py
classifications_file = "/Users/<your-username>/Downloads/classifications.csv"
subjects_file = "/Users/<your-username>/Downloads/subjects.csv"
```


#### Example 3: Data in your Google Drive

```py
# Code originally from https://github.com/kingsdigitallab/lwm-davizct
# Update the paths to match the location of files on your Google Drive
# Running this code will trigger a dialogue to allow access to your Google Drive
ipython = get_ipython()
IN_COLAB = "google.colab" in str(ipython)

if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")
    data_path = "drive/My Drive/Zooniverse_exports"
    sources_path = "drive/My Drive/Zooniverse_exports" # not needed in this notebook?
    subjects_path = "drive/My Drive/Zooniverse_exports" # not needed in this notebook?

```

## Now it is your turn!

**Fill in the file locations in this cell:**

In [1]:
classifications_file = "/Users/mridge/Desktop/Zooniverse_data_downloads_20230922/living-with-machines-classifications.csv"
subjects_file = "/Users/mridge/Desktop/Zooniverse_data_downloads_20230922/living-with-machines-subjects.csv"

---

## Setting up our definitions

**Now, we are ready to get started working with our data.**

First, we need to import the packages that we are going to use in the script below. Most of them are built-in to Python.

In [2]:
from collections import ChainMap
import hashlib
import json

import pandas as pd

## Read in our main data

Next, it's time to read in the main CSV data as a pandas DataFrame, using the handy method `.read_csv()`.

In [3]:
df = pd.read_csv(classifications_file)

Pandas has a handy method to look at the data that we have just imported, called `.head(num)`, where `num` should be replaced with the amount of rows in the frame that you want to see.

We run it below to see what our imported data looks like unprocessed:

In [4]:
df.head(3)

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,182764251,BLDigital,1744856.0,a1101d7a939fa11891dd,12038,Classify and summarise articles (set 1),16.37,2019-09-12 14:14:53 UTC,NaN,NaN,"{""source"":""api"",""session"":""5ae397cb5b23d52e566...","[{""task"":""T0"",""task_label"":""Does this article ...","{""36628096"":{""retired"":null,""!page"":""0001"",""!i...",36628096
1,182765789,BLDigital,1744856.0,a1101d7a939fa11891dd,12038,Classify and summarise articles (set 1),17.38,2019-09-12 14:19:12 UTC,NaN,NaN,"{""source"":""api"",""session"":""cf9b4c60343be171828...","[{""task"":""T0"",""task_label"":""Does this article ...","{""36627906"":{""retired"":null,""!page"":""0006"",""!i...",36627906
2,182772640,BLDigital,1744856.0,a1101d7a939fa11891dd,12051,Classify articles (set 2),9.80,2019-09-12 14:36:11 UTC,NaN,NaN,"{""source"":""api"",""session"":""4c47f81a0f62a51a2b6...","[{""task"":""T0"",""task_label"":""Does this article ...","{""36695806"":{""retired"":null,""!page"":""0008"",""!i...",36695806


As we can see in our preview, the `classification_id` is a column, which is unique for each classification. It can thus be used as an "index" for the DataFrame, a nice way of querying the frame by individual IDs.

Here's how we can set the index on our `df`:

In [5]:
df.set_index("classification_id", inplace=True)

Because Python and Pandas are both "succeeding silently", it can be good to sometimes have what we call a "reality check" to ensure that we have the result that we want.

In [6]:
df.head(3)

,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
classification_id,,,,,,,,,,,,,
182764251,BLDigital,1744856.0,a1101d7a939fa11891dd,12038,Classify and summarise articles (set 1),16.37,2019-09-12 14:14:53 UTC,NaN,NaN,"{""source"":""api"",""session"":""5ae397cb5b23d52e566...","[{""task"":""T0"",""task_label"":""Does this article ...","{""36628096"":{""retired"":null,""!page"":""0001"",""!i...",36628096
182765789,BLDigital,1744856.0,a1101d7a939fa11891dd,12038,Classify and summarise articles (set 1),17.38,2019-09-12 14:19:12 UTC,NaN,NaN,"{""source"":""api"",""session"":""cf9b4c60343be171828...","[{""task"":""T0"",""task_label"":""Does this article ...","{""36627906"":{""retired"":null,""!page"":""0006"",""!i...",36627906
182772640,BLDigital,1744856.0,a1101d7a939fa11891dd,12051,Classify articles (set 2),9.80,2019-09-12 14:36:11 UTC,NaN,NaN,"{""source"":""api"",""session"":""4c47f81a0f62a51a2b6...","[{""task"":""T0"",""task_label"":""Does this article ...","{""36695806"":{""retired"":null,""!page"":""0008"",""!i...",36695806


## Ensure correct encoding

Next, we will want to have a look at the encoding of each column.

In the "reality check" above, you can see that the columns `subject_data`, `annotations`, and `metadata` are structured as JSON (JavaScript Object Notation), which the DataFrame parser cannot interpret on its own, which is why we have to help it using the following parsing of those specific columns.

If you use Pandas' `.apply()` method (which can be applied on a column or on the entire DataFrame), you can help the software interpret the data in the columns. Here, we pass a function (`json.loads`, i.e. the `loads` function from Python's built-in `json` package) to each of the columns containing JSON data.

In [7]:
df["metadata"] = df["metadata"].apply(json.loads)
df["annotations"] = df["annotations"].apply(json.loads)

Again, we might want to do a "reality check". To look at a specific part of a dataframe we can "slice it" by passing it a list (note the double `[[` and `]]` on each side of the selector) of the columns we want to look at:

In [8]:
df[["annotations", "metadata"]]

,annotations,metadata
classification_id,,
182764251,"[{'task': 'T0', 'task_label': 'Does this artic...","{'source': 'api', 'session': '5ae397cb5b23d52e..."
182765789,"[{'task': 'T0', 'task_label': 'Does this artic...","{'source': 'api', 'session': 'cf9b4c60343be171..."
182772640,"[{'task': 'T0', 'task_label': 'Does this artic...","{'source': 'api', 'session': '4c47f81a0f62a51a..."
182773477,"[{'task': 'T0', 'task_label': 'Does this artic...","{'source': 'api', 'session': '9a94c60ab0d0adf2..."
182775776,"[{'task': 'T0', 'task_label': 'Does this artic...","{'source': 'api', 'session': 'c3057982c76d2876..."
...,...,...
494468187,"[{'task': 'T0', 'task_label': 'We searched for...","{'source': 'api', 'session': '811fa5b5f2675989..."
494468312,"[{'task': 'T0', 'task_label': 'We searched for...","{'source': 'api', 'session': '811fa5b5f2675989..."
494468466,"[{'task': 'T0', 'task_label': 'We searched for...","{'source': 'api', 'session': '811fa5b5f2675989..."


Looks good!

In the following two sections, we will process the `metadata`, and `annotations` column respectively.

## Extract classification metadata (`metadata`)

Next, we want to do something similar to what we did with the subjects above to process the metadata for each classification.

Because the data in the `metadata` column has nested JSON data (that is, objects and lists that are wrapped inside each other), we want to use the particular `json_normalize` method.

Here's an example of what one `metadata` row looks like:

In [9]:
df.metadata[df.metadata.head(1).index[0]]

{'source': 'api',
 'session': '5ae397cb5b23d52e566a93b9db18d40385b9e2a54a5bc21a5dc0670947b71eb2',
 'viewport': {'width': 1366, 'height': 654},
 'started_at': '2019-09-12T14:14:36.447Z',
 'user_agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0',
 'utc_offset': '-3600',
 'finished_at': '2019-09-12T14:14:53.177Z',
 'live_project': False,
 'interventions': {'opt_in': True, 'messageShown': False},
 'user_language': 'en',
 'user_group_ids': [],
 'subject_dimensions': [{'clientWidth': 634,
   'clientHeight': 72,
   'naturalWidth': 634,
   'naturalHeight': 72}],
 'subject_selection_state': {'retired': False,
  'selected_at': '2019-09-12T14:14:36.224Z',
  'already_seen': False,
  'selection_state': 'normal',
  'finished_workflow': False,
  'user_has_finished_workflow': False},
 'workflow_translation_id': '24214'}

Now, let's create a separate DataFrame from the normalised JSON data in the columns using the json_normalize method and use `.head()` to see the result:

In [10]:
df_metadata = pd.json_normalize(df["metadata"])

df_metadata.head(3)

,source,session,started_at,user_agent,utc_offset,finished_at,live_project,user_language,user_group_ids,subject_dimensions,...,viewport.height,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before
0,api,5ae397cb5b23d52e566a93b9db18d40385b9e2a54a5bc2...,2019-09-12T14:14:36.447Z,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69...,-3600,2019-09-12T14:14:53.177Z,False,en,[],"[{'clientWidth': 634, 'clientHeight': 72, 'nat...",...,654.0,True,False,False,2019-09-12T14:14:36.224Z,False,normal,False,False,NaN
1,api,cf9b4c60343be1718280b5dfeb8e4558398704aad25c7b...,2019-09-12T14:18:26.521Z,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69...,-3600,2019-09-12T14:19:11.829Z,False,en,[],"[{'clientWidth': 816, 'clientHeight': 440, 'na...",...,654.0,True,False,False,2019-09-12T14:18:26.462Z,False,normal,False,False,NaN
2,api,4c47f81a0f62a51a2b6b45be1efb1cc26da9a87cd4a7e0...,2019-09-12T14:35:46.499Z,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69...,-3600,2019-09-12T14:36:11.339Z,False,en,[],"[{'clientWidth': 388, 'clientHeight': 589, 'na...",...,654.0,True,False,False,2019-09-12T14:35:46.342Z,False,internal_fallback,False,False,NaN


Since we know that the shape of the classification metadata DataFrame (`df_metadata`) and the main classification DataFrame (`df`) are the same, we can apply the `set_index` method to the metadata to get the `classification_id` as index on the `df_metadata`:

In [11]:
df_metadata.set_index(df.index, inplace=True)

Reality check time again — good practice, keep track of what you're doing:

In [12]:
df_metadata.head(3)

,source,session,started_at,user_agent,utc_offset,finished_at,live_project,user_language,user_group_ids,subject_dimensions,...,viewport.height,interventions.opt_in,interventions.messageShown,subject_selection_state.retired,subject_selection_state.selected_at,subject_selection_state.already_seen,subject_selection_state.selection_state,subject_selection_state.finished_workflow,subject_selection_state.user_has_finished_workflow,seen_before
classification_id,,,,,,,,,,,,,,,,,,,,,
182764251,api,5ae397cb5b23d52e566a93b9db18d40385b9e2a54a5bc2...,2019-09-12T14:14:36.447Z,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69...,-3600,2019-09-12T14:14:53.177Z,False,en,[],"[{'clientWidth': 634, 'clientHeight': 72, 'nat...",...,654.0,True,False,False,2019-09-12T14:14:36.224Z,False,normal,False,False,NaN
182765789,api,cf9b4c60343be1718280b5dfeb8e4558398704aad25c7b...,2019-09-12T14:18:26.521Z,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69...,-3600,2019-09-12T14:19:11.829Z,False,en,[],"[{'clientWidth': 816, 'clientHeight': 440, 'na...",...,654.0,True,False,False,2019-09-12T14:18:26.462Z,False,normal,False,False,NaN
182772640,api,4c47f81a0f62a51a2b6b45be1efb1cc26da9a87cd4a7e0...,2019-09-12T14:35:46.499Z,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:69...,-3600,2019-09-12T14:36:11.339Z,False,en,[],"[{'clientWidth': 388, 'clientHeight': 589, 'na...",...,654.0,True,False,False,2019-09-12T14:35:46.342Z,False,internal_fallback,False,False,NaN


## Extract annotations (`annotations`)

The final information we want to extract and process from the main DataFrame are the annotations for each classification.

In order to do so, we must first preprocess the JSON data that we have available, extracting a list of only the information we want (the "annotation values"). We set up a function that processes each individual row, and then apply it to each row, using the `.apply` method, which we have used before:

In [13]:
def extract_annotation_values(annotation_row):
    """
    Takes an annotation row, which contains a list of tasks with values in dictionary {task, task_label, value}
    and extracts the `value` for each `task`, disregarding the `task_label` and returns them as a dictionary,
    for easy insertion into a DataFrame.
    """
    
    extracted_dictionaries = [{task_data.get('task'): task_data.get('value')} for task_data in annotation_row]
    return dict(ChainMap(*extracted_dictionaries))

df["annotations"] = df["annotations"].apply(extract_annotation_values)

Then, similarly to section 4 above, we loop through each row (of JSON data - contained in `json_data`) and extract the classification_id and annotations for each of them, which we then add on our new DataFrame `df_annotations`:

In [ ]:
df_annotations = pd.DataFrame()

for index, row in df.iterrows():
    s = pd.Series(row.annotations, name=index)
    df_annotations = pd.concat([df_annotations, s], axis=1)
    
df_annotations = df_annotations.T

Finally, we can use the `.head()` method again to check our results:

In [ ]:
df_annotations.head(3)

## Extract Zooniverse subjects (`subject_data`)

Next, we want to read in the `subjects.csv` file above, so we know what files each of the classifications were done on.

In [ ]:
df_subjects = pd.read_csv(subjects_file)

Let's have a quick look at our data using the `.head()` method that we learned above:

In [ ]:
df_subjects.head(3)

As with the classifications, we can see from this preview that the `subject_id` is a column, which is unique for each subject. Similarly to what we did with the classifications above, we can turn it into an "index" for the DataFrame, a nice way of querying the frame by individual IDs. Again, here's how we'd do it:

In [ ]:
df_subjects.set_index("subject_id", inplace=True)

Now, let's see what happened:

In [ ]:
df_subjects.head(3)

As we can see from the preview, there are some quirky aspects of the `subjects.csv` file from Zooniverse. Two things we might want to address here is that the `metadata` column is formatted as embedded JSON data, and so is the `locations` column.

The `metadata` column corresponds to the data from any manifest file that you uploaded with your subject sets, so it often contains valuable information for us to have with our classifications.

The `locations` column contains the URL to all of the images that each subject links to. For our projects, we might just have one location, but not necessarily: subjects can contain multiple images!

For our purposes, we want to create a more readable list of locations and extract the metadata into its own columns. Let's go ahead and do that now.

We will start by making sure that both columns are formatted correctly, as we did with the classifications' metadata and annotations above:

In [ ]:
df_subjects["metadata"] = df_subjects["metadata"].apply(json.loads)
df_subjects["locations"] = df_subjects["locations"].apply(json.loads)

We will start with the `locations` column. Since the data is structured as a Python dictionary `{id: "URL"}`, we can get a list of all the URLs by using Python's `dict` type's built-in `.values()` method. Here, we put that method into a custom function, which takes any row from the `subjects` DataFrame, makes a list from its values, and then joins them together with a comma separation (see the `", ".join()` syntax):

In [ ]:
def locations_as_list(row):
    return ", ".join(list(row.values()))

df_subjects["locations_list"] = df_subjects["locations"].apply(locations_as_list)

Now, to save some space, we can remove the old `locations` columns:

In [ ]:
df_subjects = df_subjects.drop(["locations"], axis="columns")

Next, we will turn to the subjects' `metadata` column.

Similarly to the extraction of classifications metadata above, we will create a separate DataFrame from the normalised JSON data from the subjects column using the `json_normalize` method and then use `.head()` to see the result:

In [ ]:
df_subjects_metadata = pd.json_normalize(df_subjects["metadata"])

df_subjects_metadata.head(3)

Again, since we know that the shape of the subjects' metadata DataFrame (`df_subjects_metadata`) and the main subject DataFrame (`df_subject`) are the same, we can apply the `set_index` method to the metadata to get the `subject_id` as index on the `df_subjects_metadata`.

After, we'll run the `.head()` reality check, to make sure all is well:

In [ ]:
df_subjects_metadata.set_index(df_subjects.index, inplace=True)

df_subjects_metadata.head(3)

Since we've now extracted the `metadata` column into a separate DataFrame, let's go ahead and remove the column from the original or "main" `df_subjects` DataFrame:

In [ ]:
df_subjects = df_subjects.drop("metadata", axis="columns")

A really great feature of DataFrames is that you can join them back together, as long as the `pandas` library has an easy way of matching the two DataFrames (or more!) together. A joined index, such as we have for `df_subjects` and `df_subjects_metadata` is a great way to do so.

In order to get one large `df_subjects` DataFrame that contains all the data for each subject, we can thus run the `.join` method:

In [ ]:
df_subjects = df_subjects.join(df_subjects_metadata)

Once again, we see that `pandas` succeeds silently, so let's do a reality check:

In [ ]:
df_subjects.head(3)

That is one large (but handy) DataFrame!

## Joining all the data back together

In this final step, we want to join the data back together into a new main DataFrame, `df_final`, from which we will remove all the personal data so the dataset can be shared publicly, if we want.

To recap, here are the four DataFrames that we have created thus far:

#### Main DataFrame (`df`)

In [ ]:
df.head(1)

#### Subjects DataFrame (`df_subjects`)

In [ ]:
df_subjects.head(1)

#### Metadata DataFrame (`df_metadata`)

In [ ]:
df_metadata.head(1)

#### Annotations DataFrame (`df_annotations`)

In [ ]:
df_annotations.head(1)

### Drop existing data

As you can see from the summary above, of the main DataFrame (`df`), it now doubles the information in `df_subjects`, `df_metadata`, and `df_annotations`. So before we do anything else, we will drop the columns that we have now processed from `df` to reduce the size of `df`:

In [ ]:
df = df.drop(["subject_data", "metadata", "annotations"], axis="columns")

### Redact personal information

Next, we want to ensure that we have anonymised the user data.

Here, we have created our own function, `redact_username`, which uses a cryptographic hash method from Python's built-in `hashlib` library called `sha256`. You can read more about [the algorithm's history on Wikipedia](https://en.wikipedia.org/wiki/SHA-2) if you are interested.

In [ ]:
def redact_username(row):
    return hashlib.sha256(str(row).encode()).hexdigest() if not pd.isna(row) else None
    
df["user_name_redacted"] = df["user_name"].apply(redact_username)

We might also be interested in knowing whether a user was logged in or not when they annotated the subject. In order to preserve that information, we create another custom function, `user_was_logged_in`, which uses the information from the `user_name` column (which has a value like `not-logged-in-98ff168ef257e2fd9d4a` if the user was not logged in) to extract a `True` or `False` value (also called a "boolean"):

In [ ]:
def user_was_logged_in(row):
    return "not-logged-in" not in row if not pd.isna(row) else False

df["user_logged_in"] = df["user_name"].apply(user_was_logged_in)

Finally, we want to drop columns that contain personal identifying information - usernames, IDs and IP addresses - from the main DataFrame:

In [ ]:
df = df.drop(["user_id", "user_name", "user_ip"], axis="columns")

### Join all the data

Now, it's time to join all the DataFrames into one new `df_final` frame.

Adding the metadata and the annotations on the main `df` is an easy thing, since all of them share index:

In [ ]:
df_final = df.join(df_metadata).join(df_annotations)

Let's do a quick reality check to make sure that all the data is there:

In [ ]:
df_final.head(1)

So far, so good. But now we want to join our subjects on the `df_final` table as well. It's not entirely as easy as the previous multiple join task that we just did here.

This time, because we're not using data points that are uniquely connected, one-to-one (one subject, in fact, will have many classifications), we will end up with duplicated information in the table. In order to join the main DataFrame with the subject DataFrame `df_subjects`, we need to make one intermediary step: We need to make both index columns available in each of the DataFrames.

_Why do we need to make `df_final`'s index a column?_ We will _merge_ two DataFrames, which is a function that will discard both indices of the merging DataFrames.

In [ ]:
df_subjects["subject_ids"] = df_subjects.index
df_final["classification_id"] = df_final.index

Before we run the merge, we also want to make sure that the two matching columns on both DataFrames are of the same (correct) type—otherwise Pandas will have trouble finding the matching subject row for each classification.

For that, we use the built-in method on each DataFrame, `.astype()` which "casts" a column as a type:

In [ ]:
df_subjects["subject_ids"] = df_subjects["subject_ids"].astype(int)
df_final["subject_ids"] = df_final["subject_ids"].astype(int)

Now we should be able to use the `merge` function from Pandas central library to join the two on the column that is shared between the two DataFrames (in our case `subject_ids`):

In [ ]:
df_merged = pd.merge(df_final, df_subjects, on="subject_ids", how="left", suffixes=("_classification", "_subject"))

Finally, we can make the `classification_id` column an index again, on our final DataFrame:

In [ ]:
df_merged = df_merged.set_index("classification_id")

One last reality check, before we're ready to export!

This time, however, we set the max column to be displayed to `None` because we're interested in seeing all the columns. Then we use our now familiar `.head()` method:

In [ ]:
pd.set_option('display.max_columns', None)
df_merged.head(3)

## Export!

In this final step, we can export the file to whatever format that we'd like to use.

In the case below, we opt for `.csv`, which is a common, open file format that can be easily opened in Microsoft Excel for further processing. It can also be imported into visualisation software, such as Power BI, or websites like ObservableHQ. On Colab, the file will be stored in a temporary directory. Once you've run the code below, click the 'folder' icon to view it:

In [ ]:
df_merged.to_csv("combined_zooniverse_data.csv")